# Validations

## Anycast IPs previous to target validation

Check if the ips previous to target for every target if the are identified as anycast

In [1]:
import ipinfo
import pandas as pd
from ast import literal_eval
from shapely import (
    from_geojson,
    to_geojson,
    Point,
)

from src.models.airport_model import AirportModel
from src.utils.common_functions import (
    json_file_to_dict,
    dict_to_json_file,
    get_list_files_in_path,
    get_nearest_airport_to_point
)
from src.utils.constants import (
    KEYS_FILEPATH,
    REPLICATION_PACKAGE_DIR,
    RESULTS_MODES
)

In [2]:
ANALYSIS_MODE=RESULTS_MODES[1]

EXPERIMENT_RESULTS_FOLDER = f"{REPLICATION_PACKAGE_DIR}/experiment_results_{ANALYSIS_MODE}"
ANALYSIS_FOLDER = f"{REPLICATION_PACKAGE_DIR}/analysis_{ANALYSIS_MODE}"
ROUTES_RESULTS_FILENAME = f"{ANALYSIS_FOLDER}/routes_results_{ANALYSIS_MODE}.csv"
IPS_PREVIOUS_TO_TARGET_CLASSIFIED_FILENAME = f"{ANALYSIS_FOLDER}/ips_previous_to_target_classified_{ANALYSIS_MODE}.json"

In [3]:
def is_ip_anycast_request(ip_to_check: str):
    access_token = json_file_to_dict(KEYS_FILEPATH)["ipinfo_token"]
    handler = ipinfo.getHandler(access_token)
    details = handler.getDetails(ip_to_check)
    #print(json.dumps(details.all, indent=4))
        
    try:
        anycast = details.anycast
    except:
        anycast = False
        
    return anycast

In [4]:
routes_df = pd.read_csv(ROUTES_RESULTS_FILENAME)
ips_previous_to_target = set()
for index, row in routes_df.iterrows():
    ips_route = set(literal_eval(row["ips_previous_to_target"]))
    ips_previous_to_target.update(ips_route)

try:
    ips_previous_to_target_classified: dict[str:bool] = json_file_to_dict(IPS_PREVIOUS_TO_TARGET_CLASSIFIED_FILENAME)
except:
    ips_previous_to_target_classified: dict[str:bool] = {}
    
for ip in ips_previous_to_target:
    if ip in ips_previous_to_target_classified.keys():
        continue
    if ip == "Indeterminate":
        continue
    is_anycast = is_ip_anycast_request(ip)
    ips_previous_to_target_classified[ip] = is_anycast
        
    dict_to_json_file(
        ips_previous_to_target_classified,
        IPS_PREVIOUS_TO_TARGET_CLASSIFIED_FILENAME
    )

    print(f"IP: {ip} // Is_Anycast: {is_anycast}")


In [5]:
ips_previous_to_target_classified = json_file_to_dict(IPS_PREVIOUS_TO_TARGET_CLASSIFIED_FILENAME)
ips_previous_to_target_anycast = set([
    ip 
    for ip in ips_previous_to_target_classified.keys()
    if ips_previous_to_target_classified[ip]
])

print(len(ips_previous_to_target))
print(len(ips_previous_to_target_classified))
print(len(ips_previous_to_target_anycast))

2204
2203
68


In [6]:
def function():
    for result_filename in get_list_files_in_path(EXPERIMENT_RESULTS_FOLDER):
        try:
            result_dict = json_file_to_dict(f"{EXPERIMENT_RESULTS_FOLDER}/{result_filename}")
        except:
            continue
    
        for hunter_result in result_dict["hunter_results"]:
            valid_ips_previous_to_target = []
            for ip in hunter_result["ips_previous_to_target"]:
                if ip["ip"] in ips_previous_to_target_anycast:
                    continue
                valid_ips_previous_to_target.append(ip)
            
            if len(valid_ips_previous_to_target) == 0:
                hunter_result["ips_previous_to_target"] = []
                hunter_result["location_result"] = {
                    "country": "Indeterminate",
                    "city": "Indeterminate",
                    "discs_intersect": False,
                    "nearest_airport": False,
                    "centroid": "",
                    "airports_countries": [],
                    "airports_cities": [],
                    "airports_intersection": []
                }
                
                if len(hunter_result["ips_previous_to_target"]) > 0:
                    print(result_filename)
                    print(hunter_result["ips_previous_to_target"])
                    print(valid_ips_previous_to_target)
                    print(hunter_result["location_result"]["country"])
                    print("Indeterminate")
                
            elif len(hunter_result["ips_previous_to_target"]) == len(valid_ips_previous_to_target):
                continue
            else:
                print(result_filename)
                print(hunter_result["ips_previous_to_target"])
                print(valid_ips_previous_to_target)
                print(hunter_result["location_result"]["country"])
                
                hunter_result["ips_previous_to_target"] = valid_ips_previous_to_target
                airport_raw = get_nearest_airport_to_point(from_geojson(hunter_result["ips_previous_to_target"][0]["location"]))
                airport = AirportModel(
                    iata_code=airport_raw["#IATA"],
                    size=airport_raw["size"],
                    name=airport_raw["name"],
                    # Longitude and Latitude for the point
                    location=Point(airport_raw["lat long"].split(" ")[1],
                                   airport_raw["lat long"].split(" ")[0]),
                    country_code=airport_raw["country_code"],
                    city_name=airport_raw["city"],
                )
                
                airports_intersection = [airport.to_dict()]
                hunter_result["location_result"] = {
                    "country": airport.country_code,
                    "city": airport.city_name,
                    "discs_intersect": False,
                    "nearest_airport": True,
                    "centroid": to_geojson(airport.location),
                    "airports_countries": [airport.country_code],
                    "airports_cities": [airport.city_name],
                    "airports_intersection": airports_intersection
                }

                print(airport.country_code)
            
        dict_to_json_file(result_dict, f"{EXPERIMENT_RESULTS_FOLDER}/{result_filename}")     
        
        
function()

172.67.75.29_mesh_20240226_21:40:09.json
[{'ip': '172.70.244.5', 'location': '{"type":"Point","coordinates":[8.6842,50.1155]}'}, {'ip': '172.67.75.29', 'location': '{"type":"Point","coordinates":[-122.3971,37.7621]}'}]
[{'ip': '172.70.244.5', 'location': '{"type":"Point","coordinates":[8.6842,50.1155]}'}]
DE
DE
35.186.205.6_mesh_20240224_06:44:56.json
[{'ip': '35.186.205.6', 'location': '{"type":"Point","coordinates":[-94.5786,39.0997]}'}, {'ip': '142.251.228.31', 'location': '{"type":"Point","coordinates":[19.0404,47.4984]}'}]
[{'ip': '142.251.228.31', 'location': '{"type":"Point","coordinates":[19.0404,47.4984]}'}]
US
HU
104.18.38.107_mesh_20240227_01:15:16.json
[{'ip': '213.184.52.32', 'location': '{"type":"Point","coordinates":[24.7535,59.437]}'}, {'ip': '104.18.38.107', 'location': '{"type":"Point","coordinates":[-122.3971,37.7621]}'}]
[{'ip': '213.184.52.32', 'location': '{"type":"Point","coordinates":[24.7535,59.437]}'}]
EE
EE
172.64.147.108_mesh_20240226_21:29:06.json
[{'ip': '